In [ ]:
import numpy as np
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Amazon_Unlocked_Mobile.csv')
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [ ]:
df1 = df.drop(['Product Name', 'Brand Name', 'Price', 'Review Votes'], axis=1)
df1['label'] = df1['Rating'].apply(lambda c: 1 if c >3 else (0 if c<3 else 'neut'))
df1 = df1.drop(['Rating'], axis=1)
df1 = df1[df1['label'] != 'neut']
df1 = df1.replace(np.nan, '', regex=True)

In [ ]:
def get_top_data(top_n):
    top_data_df_positive = df1[df1['label'] == 1].head(top_n)
    top_data_df_negative = df1[df1['label'] == 0].head(top_n)
    top_data_df_small = pd.concat([top_data_df_positive, top_data_df_negative])
    return top_data_df_small

In [ ]:
# Function call to get the top 30000 from each sentiment
df1 = get_top_data(top_n=30000)

# After selecting top few samples of each sentiment
print("After segregating and taking equal number of rows for each sentiment:")
print(df1['label'].value_counts())
df1.head(10)

After segregating and taking equal number of rows for each sentiment:
1    30000
0    30000
Name: label, dtype: int64


,Reviews,label
0,I feel so LUCKY to have found this used (phone...,1
1,"nice phone, nice up grade from my pantach revu...",1
2,Very pleased,1
3,It works good but it goes slow sometimes but i...,1
4,Great phone to replace my lost phone. The only...,1
8,I originally was using the Samsung S2 Galaxy f...,1
11,This is a great product it came after two days...,1
12,These guys are the best! I had a little situat...,1
14,Ordered this phone as a replacement for the sa...,1
16,I was able to get the phone I previously owned...,1


In [ ]:
df1 = df1.sample(frac = 1)
df1.head()

,Reviews,label
23520,The phone was in very very good condition! Eve...,1
5049,The product was exactly what I expected. Very ...,1
33847,I love this phone! Thank you so much.,1
16310,Love this iPhone 4s says my mom thank u verry ...,1
6134,I just want to tell this seller that his produ...,1


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

In [ ]:
nlp = spacy.load('en', disable=['parser', 'ner'])

In [ ]:
df1['Reviews'] = df1['Reviews'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df1['Reviews'] = df1['Reviews'].str.replace('[^\w\s]','')
stop = stopwords.words('english')
df1['Reviews'] = df1['Reviews'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [ ]:
def space(comment):
    doc = nlp(comment)
    return " ".join([token.lemma_ for token in doc])

In [ ]:
df1['Reviews']= df1['Reviews'].apply(space)

In [ ]:
df1.head()

,Reviews,label
23520,phone good condition everything work perfectly...,1
5049,product exactly expect pleased phone come orig...,1
33847,love phone thank much,1
16310,love iphone 4s say mom thank u verry much,1
6134,want tell seller product simply awesome excell...,1


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts = cv.fit_transform(df1['Reviews'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(text_counts, df1['label'], test_size=0.25, random_state=5)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
MNB = MultinomialNB()
MNB.fit(X_train, Y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
predicted = MNB.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print(confusion_matrix(Y_test,predicted))
print(classification_report(Y_test,predicted))
print("Accuracy: ", accuracy_score(Y_test, predicted))

[[6619  925]
 [ 319 7137]]
              precision    recall  f1-score   support

           0       0.95      0.88      0.91      7544
           1       0.89      0.96      0.92      7456

    accuracy                           0.92     15000
   macro avg       0.92      0.92      0.92     15000
weighted avg       0.92      0.92      0.92     15000

Accuracy:  0.9170666666666667


In [ ]:
from sklearn.linear_model import SGDClassifier
SGDC = SGDClassifier()
SGDC.fit(X_train, Y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
predictions = SGDC.predict(X_test)

In [ ]:
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))
print("Accuracy: ", accuracy_score(Y_test, predictions))

[[7102  442]
 [ 481 6975]]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      7544
           1       0.94      0.94      0.94      7456

    accuracy                           0.94     15000
   macro avg       0.94      0.94      0.94     15000
weighted avg       0.94      0.94      0.94     15000

Accuracy:  0.9384666666666667


In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier(criterion='entropy', random_state=1)
DT.fit(X_train, Y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=1, splitter='best')

In [ ]:
prediction = DT.predict(X_test)

In [ ]:
print(confusion_matrix(Y_test,prediction))
print(classification_report(Y_test,prediction))
print("Accuracy: ", accuracy_score(Y_test, prediction))

[[7093  451]
 [ 283 7173]]
              precision    recall  f1-score   support

           0       0.96      0.94      0.95      7544
           1       0.94      0.96      0.95      7456

    accuracy                           0.95     15000
   macro avg       0.95      0.95      0.95     15000
weighted avg       0.95      0.95      0.95     15000

Accuracy:  0.9510666666666666
